Use paired with data production notebook

# Initial stuff

In [58]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import urllib.request
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
## ANCHORUP
log_transformer=FunctionTransformer(func=np.log,inverse_func=np.exp,validate=False,check_inverse=False)
#validate has to be false for the nans to work, as does check_inverse
#Currently not being used

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from IPython.display import Audio
def announce_when_done(target_file='https://www.myinstants.com/media/sounds/meme-de-creditos-finales.mp3'):
    '''
    place at the end of a notebook, this will play a file when done.  I have no idea how it adds to 
    notebook memory overhead.  Hopefully not much.
    '''
    return Audio(url=target_file,autoplay=True)
import missingno as msno



In [22]:
wwdf=pd.read_csv('cleaned_data_072920_shaw.csv')

try:
    wwdf.set_index('id',inplace=True)
except:
    print('index probably already set')

In [64]:
memo=[]
#Quality of life features

# Mostly ordered in terms of apparent quality, except for location
# columns that seem to be about roughly the same thing
# treating long&lat&elevation separate from other location stuff for syntactical reasons
# similarly, water_quality_character&water_quality_abandoned are not in here as they should be used together
exclusives=[['basin','subvillage','region','region_code','district_code','lga','ward'],
 ['scheme_management','scheme_name'],
 ['extraction_type_group','extraction_type_class','extraction_type'],
 ['management','management_group'],
 ['payment_type','payment'],
 ['water_quality','quality_group'],
 ['quantity','quantity_group'],
 ['source_class','source','source_type'],
 ['waterpoint_type','waterpoint_type_group']
]

be_wary=['population','installer','subvillage','lga']

lage_sets=[c for c in wwdf.select_dtypes(exclude='number').columns if wwdf[c].nunique()>=100]

notes=['water_quality_character&water_quality_abandoned not in exc','lat&long&elev not in exc']

# The column transforming function I'll be using a good bit:

In [23]:
def prepare_cts(df,scaler,fill_value_input='none',fill_value_strategy='constant'):
    '''
    takes in df, scaler, fill_value_input,fill_value_strategy
    outputs transformers to handle imputing categorical variables and onehotencoding & doing specified numerical scaling.
    
    All data should be string or numeric. Output will be two transformeres, which will appropriately handle mixed data:
    
    imputer_transformer,encode&scale_transformer.
    
    Passing multiple scalers might work???? Why would you even do that though.  But it probably would.  But only if you're
    applying it to everything.  Otherwise, pass another column transformer after the fact and uh good luck with that lmao.
    
    the kw definitions are open and obvious, but be careful as some of the safeguards are gone.
    OneHotEncoder is fixed with unknown_values='ignore'
    '''
    print("if  you forgot to set up an assignment or save you should probably hit interrupt and do that now")
    number_cols_initial=[df.columns.get_loc(c) for c in df.select_dtypes(include='number')]
    cat_cols_initial=[df.columns.get_loc(c) for c in df.select_dtypes(exclude='number')]
    
    ctoheimp=ColumnTransformer([('impute',SimpleImputer(strategy=fill_value_strategy,
                                                        fill_value=fill_value_input),cat_cols_initial)]
                               ,remainder='passthrough')
    
    
    ctohe2=ColumnTransformer([('OHE',OneHotEncoder(handle_unknown='ignore',sparse=False),slice(0,len(cat_cols_initial))),
                              ('rescale num2',scaler,slice(len(cat_cols_initial),
                                                                                      df.shape[1]))],
                             remainder='passthrough')     
    #It'd be nice to figure out a cleaner way to handle sparse arrays
    return ctoheimp,ctohe2

# Starting with just some collection of features:
(still would like a way to do a faster/better imputing)

In [59]:
X=wwdf.copy()[['basin','extraction_type','latitude','longitude','payment_type','water_quality_character','water_quality_abandoned']]
y=wwdf.copy()['status_group']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=60120)

if  you forgot to set up an assignment or save you should probably hit interrupt and do that now


## Using a DCT model:
(need to adjust how it's scored)

In [66]:
imp,enc=prepare_cts(X_train,MinMaxScaler())

pipeline=Pipeline([('impute categorical',imp),('encode&scale',enc),('KNN impute',KNNImputer()),('DCT',DecisionTreeClassifier(random_state=60120))])

scores=cross_val_score(pipeline,X_train,y_train)

print(scores)
print('mean score:',scores.mean())

memo.append(
    
    {'name':'Vanilla DCT','model':DecisionTreeClassifier(random_state=60120),
     'model_params':pipeline['DCT'].get_params(),
     'full_params':pipeline.get_params(),
    'features':X_train.columns,
    'scores':scores,
    'mean_scores':scores.mean()}
           )

announce_when_done()


if  you forgot to set up an assignment or save you should probably hit interrupt and do that now
[0.69833754 0.69886364 0.68939394 0.69896886 0.69633838]
mean score: 0.6963804713804713


## Using a vanilla XGB, still with not-great scoring

In [ ]:
imp,enc=prepare_cts(X_train,MinMaxScaler())

pipeline=Pipeline([('impute categorical',imp),('encode&scale',enc),('KNN impute',KNNImputer()),('fxn',
                                                                                               XGBClassifier()
                                                                                               )])

scores=cross_val_score(pipeline,X_train,y_train)

print(scores)
print('mean score:',scores.mean())

memo.append(
    
    {'name':'Vanilla XGB','model':pipeline['fxn'],
     'model_params':pipeline['fxn'].get_params(),
     'full_params':pipeline.get_params(),
    'features':X_train}
           )

announce_when_done()


if  you forgot to set up an assignment or save you should probably hit interrupt and do that now
